# **Deformable Convolution for Lung Cancer Prediction Based on Transcriptomic Data : a Basic Convolutional Neural Network**
> Author : **Aymen MERROUCHE**. <br>
> In this notebook, we implement a basic CNN for our binary classification task. First we pre train our CNN on the non Lung cancer dataset. Then, we fine tune it on the lung cancer dataset (we don't freeze the convolutional layers, and we don't keep the dense layers) :

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import numpy as np
import datetime

from utils import *
from train import *
from data_utils import *
from modules.basic_cnn import *
from modules.focal_loss import *
%load_ext autoreload
%autoreload 2

In [2]:
# device to use, if cuda available then use cuda else use cpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Working on : ", device)

Working on :  cuda


In [3]:
# load hyperparametrs
# data paths args
with open('./configs/data_paths.yaml', 'r') as stream:
    data_paths_args  = yaml.load(stream,Loader=yaml.Loader)
    
    
# basic cnn args
with open('./configs/basic_cnn.yaml', 'r') as stream:
    basic_cnn_args  = yaml.load(stream,Loader=yaml.Loader)

## **1 - Pre-Training on the Non Lung Dataset :**

### **1 - 1 - Get the Data :**

In [4]:
%%time
# Getting the data
# dataset
non_lung_dataset = TranscriptomicImagesDatasetNonLung(data_paths_args["path_to_pan_cancer_hdf5_files"], data_paths_args["path_to_treemap_images"])
non_lung_dataloader_train, non_lung_dataloader_validation = get_data_loaders(non_lung_dataset, batch_size_train = basic_cnn_args["batch_size_pt"],\
                                                                             batch_size_validation = basic_cnn_args["batch_size_pt"])

CPU times: user 4.73 s, sys: 1.05 s, total: 5.78 s
Wall time: 3.01 s


### **1 - 2 - Network, Criterion and Training :**

In [5]:
# network
net = Basic_CNN().to(device).double()

# loss and optimizer  
criterion = FocalLoss().to(device)
optimizer = optim.Adam(net.parameters(), lr=basic_cnn_args['lr_pt'])

# Logging + Experiment

ignore_keys = {'no_tensorboard'}
# get hyperparameters with values in a dict
hparams = {**basic_cnn_args}
# generate a name for the experiment
expe_name = '_'.join([f"{key}={val}" for key, val in hparams.items()])
print("Experimenting with : \n \t"+expe_name)
# path where to save the model
savepath = Path('models/pre_trained_cnn_checkpt.pt')
# Tensorboard summary writer
if basic_cnn_args['no_tensorboard']:
    writer = None
else:
    writer = SummaryWriter("runs/runs"+"_"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+expe_name)
    
# start the experiment
checkpoint = CheckpointState(net, optimizer, savepath=savepath)
fit(checkpoint, criterion, non_lung_dataloader_train, non_lung_dataloader_validation, basic_cnn_args['epochs'], writer=writer)
if not basic_cnn_args['no_tensorboard']:
    writer.close()

Epoch 1/50:   0%|          | 0/41 [00:00<?, ?it/s, loss=2.3246e-01]

Experimenting with : 
 	epochs=50_batch_size_pt=128_lr_pt=0.001_batch_size_ft=128_lr_ft=0.001_no_tensorboard=True
Training on GPU 



Epoch 1/50: 100%|██████████| 41/41 [00:12<00:00,  3.22it/s, loss=1.5731e-01]
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/50, Train Loss: 1.9766e-01, Test Loss: 0.0717
Epoch 1/50, Train Accuracy: 50.30%, Test Accuracy: 88.36%
Epoch 1/50, Train AUC: 50.00%, Test AUC: 49.98%
Classification Report on Val Set : 
              precision    recall  f1-score   support

   No Cancer       0.88      1.00      0.94      2248
      Cancer       0.00      0.00      0.00       295

    accuracy                           0.88      2543
   macro avg       0.44      0.50      0.47      2543
weighted avg       0.78      0.88      0.83      2543



Epoch 2/50: 100%|██████████| 41/41 [00:12<00:00,  3.22it/s, loss=1.7521e-01]


...

Epoch 48/50, Train Loss: 8.6534e-03, Test Loss: 0.1544
Epoch 48/50, Train Accuracy: 90.50%, Test Accuracy: 69.02%
Epoch 48/50, Train AUC: 90.53%, Test AUC: 67.01%
Classification Report on Val Set : 
              precision    recall  f1-score   support

   No Cancer       0.94      0.70      0.80      2248
      Cancer       0.22      0.64      0.33       295

    accuracy                           0.69      2543
   macro avg       0.58      0.67      0.56      2543
weighted avg       0.85      0.69      0.74      2543



Epoch 49/50: 100%|██████████| 41/41 [00:12<00:00,  3.20it/s, loss=1.3152e-02]


Epoch 49/50, Train Loss: 4.5014e-03, Test Loss: 0.0920
Epoch 49/50, Train Accuracy: 94.08%, Test Accuracy: 75.25%
Epoch 49/50, Train AUC: 94.13%, Test AUC: 66.85%
Classification Report on Val Set : 
              precision    recall  f1-score   support

   No Cancer       0.93      0.78      0.85      2248
      Cancer       0.25      0.56      0.34       295

    accuracy                           0.75      2543
   macro avg       0.59      0.67      0.60      2543
weighted avg       0.85      0.75      0.79      2543



Epoch 50/50: 100%|██████████| 41/41 [00:12<00:00,  3.20it/s, loss=5.0914e-03]


Epoch 50/50, Train Loss: 8.2373e-03, Test Loss: 0.1475
Epoch 50/50, Train Accuracy: 91.26%, Test Accuracy: 68.56%
Epoch 50/50, Train AUC: 91.23%, Test AUC: 67.04%
Classification Report on Val Set : 
              precision    recall  f1-score   support

   No Cancer       0.94      0.69      0.79      2248
      Cancer       0.22      0.65      0.32       295

    accuracy                           0.69      2543
   macro avg       0.58      0.67      0.56      2543
weighted avg       0.85      0.69      0.74      2543


Finished.
Best validation AUC: 6.8505e-01
With accuracy: 0.7245565891265869


## **2 - Fine-Tuning on the Lung dataset :**

### **2 - 1 - Load Pre-Trained Model :**

In [6]:
# Load the pretrained Model
net = Basic_CNN().to(device).double()
optimizer = optim.Adam(net.parameters(), lr=basic_cnn_args['lr_ft'])
# path where the pre-trained model is saved : defined above+"_best"
savepath = Path('models/pre_trained_cnn_checkpt_best.pt')
checkpoint = CheckpointState(net, optimizer, savepath=savepath)
checkpoint.load()
pretrained = checkpoint.model

### **2 - 2 - Get the Data :**

In [7]:
%%time
# Getting the data
# dataset
lung_dataset = TranscriptomicImagesDatasetLung(data_paths_args["path_to_pan_cancer_hdf5_files"], data_paths_args["path_to_treemap_images"])
lung_dataloader_train, lung_dataloader_validation = get_data_loaders(lung_dataset, batch_size_train = basic_cnn_args["batch_size_ft"],\
                                                                             batch_size_validation = basic_cnn_args["batch_size_ft"])

CPU times: user 617 ms, sys: 92.4 ms, total: 710 ms
Wall time: 328 ms


### **2 - 3 - Fine Tuning Procedure :**

In [8]:
# Beginnig Of Transfer Learnig Procedure
net = fine_tune_CNN(pretrained)
net = net.to(device).double()
criterion = FocalLoss().to(device)
optimizer = optim.Adam(net.parameters(), lr=basic_cnn_args['lr_ft'])
savepath = Path('models_finetuned/fine_tuned_cnn_checkpt.pt')
checkpoint = CheckpointState(net, optimizer, savepath=savepath)
fit(checkpoint, criterion, lung_dataloader_train, lung_dataloader_validation, basic_cnn_args['epochs'])

Epoch 1/50:   0%|          | 0/5 [00:00<?, ?it/s, loss=3.1211e-01]

Training on GPU 



Epoch 1/50: 100%|██████████| 5/5 [00:01<00:00,  3.62it/s, loss=1.8677e-01]


Epoch 1/50, Train Loss: 5.1267e-02, Test Loss: 0.1548
Epoch 1/50, Train Accuracy: 84.05%, Test Accuracy: 67.69%
Epoch 1/50, Train AUC: 83.82%, Test AUC: 61.23%
Classification Report on Val Set : 
              precision    recall  f1-score   support

   No Cancer       0.92      0.76      0.83       252
      Cancer       0.19      0.47      0.27        30

    accuracy                           0.73       282
   macro avg       0.55      0.61      0.55       282
weighted avg       0.84      0.73      0.77       282



Epoch 2/50: 100%|██████████| 5/5 [00:01<00:00,  3.61it/s, loss=7.3099e-02]


...

Epoch 48/50, Train Loss: 5.8823e-04, Test Loss: 0.0991
Epoch 48/50, Train Accuracy: 99.22%, Test Accuracy: 81.65%
Epoch 48/50, Train AUC: 99.17%, Test AUC: 62.10%
Classification Report on Val Set : 
              precision    recall  f1-score   support

   No Cancer       0.92      0.91      0.91       252
      Cancer       0.30      0.33      0.32        30

    accuracy                           0.85       282
   macro avg       0.61      0.62      0.62       282
weighted avg       0.85      0.85      0.85       282



Epoch 49/50: 100%|██████████| 5/5 [00:01<00:00,  3.61it/s, loss=6.9748e-04]


Epoch 49/50, Train Loss: 2.4685e-04, Test Loss: 0.0968
Epoch 49/50, Train Accuracy: 99.84%, Test Accuracy: 81.13%
Epoch 49/50, Train AUC: 99.83%, Test AUC: 57.30%
Classification Report on Val Set : 
              precision    recall  f1-score   support

   No Cancer       0.91      0.91      0.91       252
      Cancer       0.24      0.23      0.24        30

    accuracy                           0.84       282
   macro avg       0.58      0.57      0.57       282
weighted avg       0.84      0.84      0.84       282



Epoch 50/50: 100%|██████████| 5/5 [00:01<00:00,  3.61it/s, loss=3.8337e-04]


Epoch 50/50, Train Loss: 2.9948e-04, Test Loss: 0.0972
Epoch 50/50, Train Accuracy: 99.22%, Test Accuracy: 81.39%
Epoch 50/50, Train AUC: 99.17%, Test AUC: 58.97%
Classification Report on Val Set : 
              precision    recall  f1-score   support

   No Cancer       0.91      0.91      0.91       252
      Cancer       0.27      0.27      0.27        30

    accuracy                           0.84       282
   macro avg       0.59      0.59      0.59       282
weighted avg       0.84      0.84      0.84       282


Finished.
Best validation AUC: 6.6032e-01
With accuracy: 0.7025240461031595
